# Plant_Seedlings_Classification

Applying what I learnt so far in fastai in the Kaggle Competetions, was really a fun one indeed. Got stuck while creating a csv file for the submission but somehow with the help of people's in forums got through. 

The Competetions link: https://www.kaggle.com/c/plant-seedlings-classification/overview

In [ ]:
from fastai.vision.all import *
from fastai import * 

In [ ]:
# The path to the data 

train_data = Path('train/')
test_data = Path('test/')

In [ ]:
get_image_files_sorted??

In [ ]:
# Getting the filenames 

train_fns = get_image_files(train_data)
train_fns

test_fns = get_image_files(test_data)
test_fns

In [ ]:
# Building a Data Block 

plant = DataBlock(blocks=(ImageBlock , CategoryBlock) , 
                   get_items=get_image_files,
                   get_y = parent_label, 
                   item_tfms = RandomResizedCrop(460, min_scale=0.75),
                   batch_tfms = [*aug_transforms(size=224, max_warp=0 , max_zoom= 0.8 ), Normalize.from_stats(*imagenet_stats)])

# Creating a Dataloader 

train_dls = plant.dataloaders(train_data , bs = 64)

# Checking how well our data has been formatted 
train_dls.show_batch()

In [ ]:
# Getting the Metrics for evaluation 

f1_score = F1Score(average='micro')
precision = Precision(average='micro')
recall = Recall(average='micro')

In [ ]:
# Creating a model 

learn = cnn_learner(train_dls, resnet50, pretrained=True, metrics=[error_rate ,f1_score, precision , recall ,accuracy]).to_fp16()

In [ ]:
# Finding a learnin rate / freeze 
learn.lr_find()

In [ ]:
# Running for 3 epochs / freezed other layers 

learn.fit_one_cycle(3 , 4e-3)

In [ ]:
# Unfreezing other network 
learn.unfreeze()

In [ ]:
# Again wanna find the learning rate , because we unfreezed all the networks 

learn.lr_find()

In [ ]:
# Training the whole network for 10 epochs 

learn.fit_one_cycle(10 , lr_max = slice(6e-5 , 1e-4))

First, I wanna make a submission in Kaggle then let's worry about improving the model

In [ ]:
# Checking the results 
learn.show_results(figsize=(12,8))

In [ ]:
# Confusion Matrix 

interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix(figsize=(16,12))

In [ ]:
# Creating a test dataloader 

test_dl = train_dls.test_dl(test_fns)

# Getting the predictions for the test set 

test_pred = learn.get_preds(dl = test_dl)
test_pred

In [ ]:
# Getting in the Sample Submission File 
sample_sub = pd.read_csv('sample_submission.csv')
sample_sub

**CAUTION** Below function didn't work for this problem, still it can work for problems like Dog Breed Identification. 

In [ ]:
# Let's write helper function for this 

def create_predictions_csv(test_preds, sample_submission):
    
    '''
    INPUT --> Takes in test predictions from a test dataloader, with tuple format and the sample submission csv file
    
    OUTPUT --> Return the test dataframe of results
    
    '''
    
    import pandas as pd 
    sample_sub = sample_submission
    test_pred = test_preds[0].numpy()
    test_df = pd.DataFrame(test_pred)
    
    df_test = pd.concat([sample_sub['file'],test_df],axis=1)
    return df_test
                        
    
df_test = create_predictions_csv(test_preds=test_pred , sample_submission=sample_sub)    
df_test    
    

In [ ]:
test_d = df_test.drop('file' , axis=1 )


In [ ]:
t_b = df_test.drop('file' , axis=1 )

In [ ]:
# COnverting to numpy 
test_d = test_d.to_numpy()

In [ ]:
# Checking the index

tr_list = list(train_dls.vocab)
idx = 0
for i in tr_list:
    print(f'{i}: {idx}')
    idx+= 1

In [ ]:
mapping={0: 'Black-grass',
 1: 'Charlock',
 2: 'Cleavers',
 3: 'Common Chickweed',
 4: 'Common wheat',
 5 : 'Fat Hen',
 6 : 'Loose Silky-bent',
 7 : 'Maize',
 8 : 'Scentless Mayweed',
 9 : 'Shepherds Purse',
 10 : 'Small-flowered Cranesbill',
 11: 'Sugar beet'}
mapping

In [ ]:
col_df = col_df.rename(columns=mapping)
df_test = pd.concat([df_test['file'] , col_df] , axis = 1)
df_test

In [ ]:
col_df = df_test.drop('file' , axis=1)

In [ ]:
columns_list = []
for col in col_df.columns:
    columns_list.append(col)
columns_list    

In [ ]:
import numpy as np

In [ ]:
real_pred = np.argmax(test_d , axis = 1)
real_pred

In [ ]:
test_dl.items

In [ ]:
# Creating csv in 3 lines, learnt a new trick

test_pred_max = test_pred[0].argmax(dim=1)
pred_labels = [train_dls.vocab[o] for o in test_pred_max]
sub = pd.DataFrame({'file':test_dl.items,'species':pred_labels})
sub

In [ ]:
sub['file'] = sub['file'].astype(str).str.replace('test/' , '')
sub

In [ ]:
sub.to_csv('submission.csv' , index=False)

Was able to bag `97%` accuracy with just one shot, at first there was filenames mismatch which pulled my score off very low. After going on a hunt googling and checking on forums, I was able to solve this from a member (Issac) in the forum. 

I learnt how to ask clear questions which could results in the answer we been looking for. 